In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [9]:
data = pd.read_csv("Churn_Modelling.csv")


label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])

onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]])

geo_encoded_df = pd.DataFrame(
    geo_encoder.toarray(),
    columns=onehot_encoder_geo.get_feature_names_out(["Geography"]),
)

final_df = pd.concat([data.drop("Geography", axis=1), geo_encoded_df], axis=1)

final_df = final_df.drop(
    ["RowNumber", "CustomerId", "Surname"],
    axis=1,
)
X = final_df.drop("Exited", axis=1)
y = final_df["Exited"]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation="relu", input_shape=(X_train.shape[1],)))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation="relu"))

    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [22]:
model = KerasClassifier(
    build_fn=create_model, neurons=32, layers=1, epochs=50, batch_size=10, verbose=0
)

In [23]:
param_grid = {
    "neurons": [16, 32, 64, 128],
    "layers": [1, 2, 3],
    "epochs": [50, 100],
}

In [24]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

c:\Users\SahilRaval\Desktop\Code\Personal\ANN Classification\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\SahilRaval\Desktop\Code\Personal\ANN Classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best: 0.8586243861302784 using {'epochs': 50, 'layers': 2, 'neurons': 16}
